In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ast
import pickle
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import keras
from keras.models import Sequential
from keras.layers import Input, Embedding, Conv1D, MaxPool1D, GlobalMaxPooling1D, Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
import keras_tuner as kt
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
EPOCHS=500
BATCH_SIZE=128
PATIENCE=5

In [4]:
# base_path = '/content/drive/MyDrive/fake_news/'
base_path = ''

# Load Embeddings & Inputs

In [5]:
with open(base_path+'artifacts/embeddings_inputs.pkl', 'rb') as f:
    loaded_input_items = pickle.load(f)

In [6]:
embedding_matrix = loaded_input_items['embedding_matrix']
X_train_pad = loaded_input_items['X_train_pad']
X_val_pad = loaded_input_items['X_val_pad']
X_test_pad = loaded_input_items['X_test_pad']
y_train = loaded_input_items['y_train']
y_val = loaded_input_items['y_val']
y_test = loaded_input_items['y_test']

In [7]:
VOCAB_SIZE = embedding_matrix.shape[0]
EMBEDDING_DIM = embedding_matrix.shape[1]
MAX_LEN = len(X_train_pad[0])

In [8]:
print(f"Embedding dimension: {EMBEDDING_DIM}\nVocab size: {VOCAB_SIZE}\nMaximum input length: {MAX_LEN}")

Embedding dimension: 500
Vocab size: 35756
Maximum input length: 588


# Tuning CNN - 1 Convolution layer

In [10]:
def build_model1(hp):
    model = Sequential()
    model.add(Input(shape=(MAX_LEN,)))
    model.add(Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=hp.Choice('embeddings_trainable', values=[True, False])))
    
    model.add(Conv1D(filters=hp.Choice(f'conv1_filters', values=[64, 128, 256, 512]),
                         kernel_size=hp.Int(f'conv1_kernel_size', min_value=2, max_value=5, step=1),
                         strides=1, padding='valid', activation='relu'))

    model.add(GlobalMaxPooling1D())

    model.add(Dense(units=hp.Choice(f'dense_units', values=[16, 32, 64, 128]),
                        activation='relu'))
    model.add(Dropout(hp.Float(f'dense_dropout', min_value=0.2, max_value=0.5, step=0.1)))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=[0.001, 0.005, 0.01, 0.05, 0.1])),
                      loss='binary_crossentropy',
                      metrics=['accuracy'])
    return model

In [13]:
# tuner = kt.Hyperband(
#     build_model,
#     objective='val_accuracy',
#     max_epochs=EPOCHS,
#     factor=3,
#     directory='tuner_results',
#     project_name='cnn_hyperband')
tuner = kt.RandomSearch(
    build_model1,
    objective='val_accuracy',
    max_trials=33,
    executions_per_trial=1,
    directory=base_path+'tuner_results',
    project_name='cnn1_randomSearch')

Reloading Tuner from tuner_results\cnn1_randomSearch\tuner0.json


In [14]:
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=PATIENCE,
                      restore_best_weights=True, verbose=1)
tuner.search(X_train_pad, y_train,
             validation_data=(X_val_pad, y_val),
             epochs=EPOCHS, batch_size=BATCH_SIZE,
             callbacks=[estop], verbose=1)

Trial 33 Complete [00h 02m 18s]
val_accuracy: 0.5054744482040405

Best val_accuracy So Far: 0.9656326174736023
Total elapsed time: 16h 06m 48s


In [15]:
best_model1 = tuner.get_best_models(num_models=1)[0]
best_hp1 = tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hp1.values)

{'embeddings_trainable': 1, 'conv1_filters': 512, 'conv1_kernel_size': 4, 'dense_units': 16, 'dense_dropout': 0.30000000000000004, 'learning_rate': 0.001}


C:\Users\aleen\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 16 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [16]:
best_loss1, best_accuracy1 = best_model1.evaluate(X_test_pad, y_test, verbose=0)
print("Best CNN Accuracy:", best_accuracy1)
print("Best CNN Loss:", best_loss1)

Best CNN Accuracy: 0.9627737402915955
Best CNN Loss: 0.12475183606147766


# Tuning CNN - 2 Convolution layers

In [17]:
def build_model2(hp):
    model = Sequential()
    model.add(Input(shape=(MAX_LEN,)))
    model.add(Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=hp.Choice('embeddings_trainable', values=[True, False])))
    
    model.add(Conv1D(filters=hp.Choice(f'conv1_filters', values=[64, 128, 256, 512]),
                         kernel_size=hp.Int(f'conv1_kernel_size', min_value=2, max_value=5, step=1),
                         strides=1, padding='valid', activation='relu'))
    model.add(MaxPool1D(pool_size=2))

    model.add(Conv1D(filters=hp.Choice(f'conv2_filters', values=[64, 128, 256, 512]),
                         kernel_size=hp.Int(f'conv2_kernel_size', min_value=2, max_value=5, step=1),
                         strides=1, padding='valid', activation='relu'))

    model.add(GlobalMaxPooling1D())

    model.add(Dense(units=hp.Choice(f'dense_units', values=[16, 32, 64, 128]),
                        activation='relu'))
    model.add(Dropout(hp.Float(f'dense_dropout', min_value=0.2, max_value=0.5, step=0.1)))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=[0.001, 0.005, 0.01, 0.05, 0.1])),
                      loss='binary_crossentropy',
                      metrics=['accuracy'])
    return model

In [18]:
tuner = kt.RandomSearch(
    build_model2,
    objective='val_accuracy',
    max_trials=33,
    executions_per_trial=1,
    directory=base_path+'tuner_results',
    project_name='cnn2_randomSearch')

In [19]:
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=PATIENCE,
                      restore_best_weights=True, verbose=1)
tuner.search(X_train_pad, y_train,
             validation_data=(X_val_pad, y_val),
             epochs=EPOCHS, batch_size=BATCH_SIZE,
             callbacks=[estop], verbose=1)

Trial 33 Complete [00h 00m 54s]
val_accuracy: 0.5054744482040405

Best val_accuracy So Far: 0.9619829654693604
Total elapsed time: 03h 42m 02s


In [20]:
best_model2 = tuner.get_best_models(num_models=1)[0]
best_hp2 = tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hp2.values)

{'embeddings_trainable': 0, 'conv1_filters': 256, 'conv1_kernel_size': 5, 'conv2_filters': 512, 'conv2_kernel_size': 5, 'dense_units': 128, 'dense_dropout': 0.4, 'learning_rate': 0.001}


C:\Users\aleen\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [21]:
best_loss2, best_accuracy2 = best_model2.evaluate(X_test_pad, y_test, verbose=0)
print("Best CNN Accuracy:", best_accuracy2)
print("Best CNN Loss:", best_loss2)

Best CNN Accuracy: 0.9583941698074341
Best CNN Loss: 0.20375950634479523


# Tuning CNN - 3 Convolution layers

In [22]:
def build_model3(hp):
    model = Sequential()
    model.add(Input(shape=(MAX_LEN,)))
    model.add(Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=hp.Choice('embeddings_trainable', values=[True, False])))
    
    model.add(Conv1D(filters=hp.Choice(f'conv1_filters', values=[64, 128, 256, 512]),
                         kernel_size=hp.Int(f'conv1_kernel_size', min_value=2, max_value=5, step=1),
                         strides=1, padding='valid', activation='relu'))
    model.add(MaxPool1D(pool_size=2))

    model.add(Conv1D(filters=hp.Choice(f'conv2_filters', values=[64, 128, 256, 512]),
                         kernel_size=hp.Int(f'conv2_kernel_size', min_value=2, max_value=5, step=1),
                         strides=1, padding='valid', activation='relu'))
    model.add(MaxPool1D(pool_size=2))

    model.add(Conv1D(filters=hp.Choice(f'conv3_filters', values=[64, 128, 256, 512]),
                         kernel_size=hp.Int(f'conv3_kernel_size', min_value=2, max_value=5, step=1),
                         strides=1, padding='valid', activation='relu'))

    model.add(GlobalMaxPooling1D())

    model.add(Dense(units=hp.Choice(f'dense_units', values=[16, 32, 64, 128]),
                        activation='relu'))
    model.add(Dropout(hp.Float(f'dense_dropout', min_value=0.2, max_value=0.5, step=0.1)))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=[0.001, 0.005, 0.01, 0.05, 0.1])),
                      loss='binary_crossentropy',
                      metrics=['accuracy'])
    return model

In [23]:
tuner = kt.RandomSearch(
    build_model3,
    objective='val_accuracy',
    max_trials=33,
    executions_per_trial=1,
    directory=base_path+'tuner_results',
    project_name='cnn3_randomSearch')

In [24]:
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=PATIENCE,
                      restore_best_weights=True, verbose=1)
tuner.search(X_train_pad, y_train,
             validation_data=(X_val_pad, y_val),
             epochs=EPOCHS, batch_size=BATCH_SIZE,
             callbacks=[estop], verbose=1)

Trial 33 Complete [00h 06m 27s]
val_accuracy: 0.94525545835495

Best val_accuracy So Far: 0.9586374759674072
Total elapsed time: 03h 33m 30s


In [25]:
best_model3 = tuner.get_best_models(num_models=1)[0]
best_hp3 = tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hp3.values)

{'embeddings_trainable': 0, 'conv1_filters': 128, 'conv1_kernel_size': 5, 'conv2_filters': 128, 'conv2_kernel_size': 3, 'conv3_filters': 512, 'conv3_kernel_size': 4, 'dense_units': 16, 'dense_dropout': 0.30000000000000004, 'learning_rate': 0.001}


C:\Users\aleen\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [26]:
best_loss3, best_accuracy3 = best_model3.evaluate(X_test_pad, y_test, verbose=0)
print("Best CNN Accuracy:", best_accuracy3)
print("Best CNN Loss:", best_loss3)

Best CNN Accuracy: 0.9552311301231384
Best CNN Loss: 0.12675246596336365


In [30]:
bests = {'models':[best_model1, best_model2, best_model3],
        'accuracies':[best_accuracy1, best_accuracy2, best_accuracy3]}
n = np.argmax(bests['accuracies'])
print(f"Best number of convulation layers: {n+1}")
best_model = bests['models'][n]

Best number of convulation layers: 1


In [31]:
best_model.save(base_path+'artifacts/tuned_cnn.keras')

In [35]:
print(keras.models.load_model(base_path+'artifacts/tuned_cnn.keras').evaluate(X_test_pad, y_test, verbose=0)[1])

C:\Users\aleen\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 16 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


0.9627737402915955
